# More Postgres + Pandas

### Instructions

* Create a `customer_db` database in pgAdmin 4 then create the following two tables within:

  * A `premise` table that contains the columns `id`, `premise_name` and `county_id`.

  * A `county` table that contains the columns `id`, `county_name`, `license_count` and `county_id`.

  * Be sure to assign a primary key, as Pandas will not be able to do so.

* In Jupyter Notebook perform all ETL.

* **Extraction**

  * Put each CSV into a pandas DataFrame.

* **Transform**

  * Copy only the columns needed into a new DataFrame.

  * Rename columns to fit the tables created in the database.

  * Handle any duplicates. **HINT:** some locations have the same name but each license number is unique.

  * Set index to the previously created primary key.

* **Load**

  * Create a connection to database.

  * Check for a successful connection to the database and confirm that the tables have been created.

  * Append DataFrames to tables. Be sure to use the index set earlier.

* Confirm successful **Load** by querying database.

* Join the two tables and select the `id` and `premise_name` from the `premise` table and `county_name` from the `county` table.


In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os

### Read *NBA_Player_Stats* CSV

In [3]:
stats = os.path.join("NBA_PLayers_Stats.csv")
stats_df = pd.read_csv(stats)
stats_df.head()

,Unnamed: 0,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,...,STL,BLK,TOV,PF,PTS,id,full_name,first_name,last_name,is_active
0,0,76001,1990-91,0,1610612757,POR,23.0,43,0.0,290.0,...,4.0,12.0,22.0,39,135,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,1,76001,1991-92,0,1610612757,POR,24.0,71,1.0,934.0,...,25.0,16.0,66.0,132,432,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
2,2,76001,1992-93,0,1610612749,MIL,25.0,12,0.0,159.0,...,6.0,4.0,13.0,24,64,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
3,3,76001,1992-93,0,1610612738,BOS,25.0,63,52.0,1152.0,...,19.0,22.0,84.0,165,514,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
4,4,76001,1992-93,0,0,TOT,25.0,75,52.0,1311.0,...,25.0,26.0,97.0,189,578,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False


In [5]:
#remove unnamed index column
del stats_df['Unnamed: 0']
stats_df.head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,AST,STL,BLK,TOV,PF,PTS,full_name,first_name,last_name,is_active
0,76001,1990-91,0,1610612757,POR,23.0,43,0.0,290.0,55,...,12,4.0,12.0,22.0,39,135,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76001,1991-92,0,1610612757,POR,24.0,71,1.0,934.0,178,...,30,25.0,16.0,66.0,132,432,Alaa Abdelnaby,Alaa,Abdelnaby,False
2,76001,1992-93,0,1610612749,MIL,25.0,12,0.0,159.0,26,...,10,6.0,4.0,13.0,24,64,Alaa Abdelnaby,Alaa,Abdelnaby,False
3,76001,1992-93,0,1610612738,BOS,25.0,63,52.0,1152.0,219,...,17,19.0,22.0,84.0,165,514,Alaa Abdelnaby,Alaa,Abdelnaby,False
4,76001,1992-93,0,0,TOT,25.0,75,52.0,1311.0,245,...,27,25.0,26.0,97.0,189,578,Alaa Abdelnaby,Alaa,Abdelnaby,False


### Read *salary* CSV

### Transform NBA stats DataFrame

In [13]:
# Rename the column headers to lowercase
stats_df.columns= stats_df.columns.str.lower()

stats_df.head()


,player_id,season_id,league_id,team_id,team_abbreviation,player_age,gp,gs,min,fgm,...,ast,stl,blk,tov,pf,pts,full_name,first_name,last_name,is_active
0,76001,1990-91,0,1610612757,POR,23.0,43,0.0,290.0,55,...,12,4.0,12.0,22.0,39,135,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76001,1991-92,0,1610612757,POR,24.0,71,1.0,934.0,178,...,30,25.0,16.0,66.0,132,432,Alaa Abdelnaby,Alaa,Abdelnaby,False
2,76001,1992-93,0,1610612749,MIL,25.0,12,0.0,159.0,26,...,10,6.0,4.0,13.0,24,64,Alaa Abdelnaby,Alaa,Abdelnaby,False
3,76001,1992-93,0,1610612738,BOS,25.0,63,52.0,1152.0,219,...,17,19.0,22.0,84.0,165,514,Alaa Abdelnaby,Alaa,Abdelnaby,False
4,76001,1992-93,0,0,TOT,25.0,75,52.0,1311.0,245,...,27,25.0,26.0,97.0,189,578,Alaa Abdelnaby,Alaa,Abdelnaby,False


### Transform Salary DataFrame

### Create database connection

In [8]:
pg_user = 'postgres'
pg_password = 'password'
db_name = 'etl_practice'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

### Confirm tables

In [9]:
engine.table_names()

['practice_nba']

### Load DataFrames into database

In [15]:
stats_df.to_sql(name='practice_nba', con=engine, if_exists='append', index=False)

In [ ]:
# salary_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)